In [1]:
from glob import glob
import skimage.io as sk
import cv2
import pickle
from skimage.color import rgb2gray
from skimage.transform import resize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,Input,SpatialDropout1D,GaussianDropout,BatchNormalization,Flatten,Add,LayerNormalization,UnitNormalization,AlphaDropout
from keras.models import Model
from keras.applications import DenseNet121
from keras.utils import to_categorical
from keras.models import Model
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from keras.backend import clear_session
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from keras.optimizers import Adam,Adadelta,Adamax,RMSprop,adafactor,SGD
from keras.regularizers import l2

2023-03-21 19:21:17.234717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 19:21:18.091558: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hrithikpaul/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-03-21 19:21:18.091601: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-21 19:21:21.131972: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; d

In [4]:
data=pd.read_csv("data1.csv")
class_val=data["class"]
col=data.columns
print(col)
features_val=data[col[0:100]]
print(features_val.shape)

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '91', '92', '93', '94', '95', '96', '97', '98', '99', 'class'],
      dtype='object', length=102)
(834, 100)


In [4]:
features_val=np.array(features_val).reshape(834,100)

In [5]:
fx=MinMaxScaler()
X=fx.fit_transform(features_val)

In [6]:
sx=StandardScaler()
X=sx.fit_transform(features_val)

In [7]:

lb=LabelEncoder()
x11=to_categorical(lb.fit_transform(class_val))
print(x11)


[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [8]:
X=np.array(X)
x11=np.array(x11)
np.random.seed(1000)
print(x11.shape)

(834, 4)


In [9]:
#features=np.array(features).reshape(145,100,1)
x_train,x_test,y_train,y_test=train_test_split(X,x11,test_size=0.2,random_state=42)
print(x_train.shape)

(667, 100)


In [10]:
from __future__ import print_function
import os
#import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
import pandas as pd

import numpy as np
import cv2
import random


# train with the train folder images
DATADIR = "./train/"
CATEGORIES = ["Hitesh", "Labhesh", "Tarun", "Varkha"] #subjects/classes to be recognized
"""
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img),cv2.IMREAD_GRAYSCALE)   # convert in gray scale for faster computation
        break
    break
"""
path_file=pd.read_csv("names.csv")
#print(path_file['0'].values)
for i in path_file['0'].values:
    #print(i)
    img_array=cv2.imread(i,cv2.IMREAD_GRAYSCALE)


IMG_WIDTH = 60      # small size images (not more than 200 * 200)
IMG_HEIGHT = 100
new_array = cv2.resize(img_array, (IMG_WIDTH, IMG_HEIGHT))

training_data = []

def create_training_data():
    """
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)  # assign distinct index to all classes
    """

    for img in path_file['0'].values:
        try:
            if "Hitesh" in img:
                class_num=0
            elif "Labhesh" in img:
                class_num=1
            elif "Tarun" in img:
                class_num=2
            else:
                class_num=3
            img_array = cv2.imread(img,cv2.IMREAD_GRAYSCALE) # convert in gray scale for faster computation
            new_array = cv2.resize(img_array, (IMG_WIDTH, IMG_HEIGHT))  # make all images of standard/same size
            training_data.append([new_array, class_num])    # assign distinct index to all classes
        except Exception as e:
            pass

print("Training Data")
create_training_data()
print(len(training_data))

random.shuffle(training_data)   # shuffle for better training and learning of the machine

a = []      #feature set
b = []      #labels

for features, labels in training_data:
    a.append(features)
    b.append(labels)

a = np.array(a).reshape(-1, IMG_WIDTH, IMG_HEIGHT, 1)
print(a.shape)
np.random.seed(1000)

X_train = a[:834]
Y_train = b[:834]

X_train = X_train/255.0
Y_train=to_categorical(Y_train)
print(X_train.shape,Y_train.shape)

# increase the epochs or decrease the batch size according to classes



Training Data
880
(880, 60, 100, 1)
(834, 60, 100, 1) (834, 4)


In [11]:
X_train=X_train.reshape(834,60*100*1)
print(X_train.shape,Y_train.shape)

(834, 6000) (834, 4)


In [12]:
from sklearn.model_selection import train_test_split
x1,x2,y1,y2=train_test_split(X_train,Y_train,test_size=0.2,random_state=42)

In [13]:
clear_session()
#model=Sequential()
#model.add(Input(shape=(100)))
"""
model.add(Dense(256,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(256,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(156,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(256,activation="relu",kernel_initializer="he_uniform"))
model.add(BatchNormalization())
model.add(Dense(256,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(512,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(128,activation="relu",kernel_initializer="he_uniform"))
model.add(BatchNormalization())
model.add(Dense(2056,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(1024,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(256,activation="relu",kernel_initializer="he_uniform"))
"""
#model.add(Flatten(input_shape=(100,)))
model=Input(shape=(100,))
#model1=(Dense(64,activation="relu",input_dim=100))(model)
#model.add(AlphaDropout(0.1))
model=(Flatten())(model)
"""
#model.add(Dense(32,activation="selu"))
model.add(Dense(64,activation="relu"))
model.add(GaussianDropout(0.1))

model.add(Dense(4,activation="softmax"))

adam=Adam(learning_rate=0.001)
model.compile(optimizer=adam,loss="binary_crossentropy",metrics=["accuracy"])
model.summary()
"""

'\n#model.add(Dense(32,activation="selu"))\nmodel.add(Dense(64,activation="relu"))\nmodel.add(GaussianDropout(0.1))\n\nmodel.add(Dense(4,activation="softmax"))\n\nadam=Adam(learning_rate=0.001)\nmodel.compile(optimizer=adam,loss="binary_crossentropy",metrics=["accuracy"])\nmodel.summary()\n'

In [14]:
clear_session()
#model1 = Sequential()
IMG_WIDTH = 60      # small size images (not more than 200 * 200)
IMG_HEIGHT = 100
model11=Input(shape=(IMG_WIDTH, IMG_HEIGHT, 1))
#model12=(Conv2D(32, kernel_size=4, strides=1,activation='relu'))(model11)
model11=Flatten()(model11)

In [15]:
from keras.layers import concatenate
model2=concatenate([model,model11])

In [17]:
clear_session()
#model.add(Conv2D(32, kernel_size=4, strides=1,activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 1)))
model2=(Conv2D(32, kernel_size=4, strides=2,activation='relu'))(model2)
model2=(MaxPooling2D(pool_size=2))(model2)
model2=(Conv2D(64, kernel_size=4, strides=1,activation='relu'))(model2)
model2=(MaxPooling2D(pool_size=2))(model2)
model2=Dense(1000,activation="relu")(model2)
model2=BatchNormalization()(model2)
model2=Dense(500,activation="relu")(model2)
mocel2=Dropout(0.2)(model2)
model2=Flatten()(model2)
model2=Dense(64,activation="relu")(model2)
x=Dense(4,activation="softmax")(model2)


ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 6100)

In [ ]:
model = Model(inputs=[model,model11], outputs=x)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 6000)]       0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 6100)         0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1000)         6101000     ['concatenate[1][0]']        

In [ ]:
#gru
model.fit([x_train,x1],y_train,verbose=1,batch_size=32,epochs=100)

Epoch 1/100
21/21 [==============================] - 2s 91ms/step - loss: 0.5841 - accuracy: 0.3043
Epoch 2/100
21/21 [==============================] - 2s 91ms/step - loss: 0.5779 - accuracy: 0.3118
Epoch 3/100
21/21 [==============================] - 2s 88ms/step - loss: 0.5732 - accuracy: 0.3388
Epoch 4/100
21/21 [==============================] - 2s 95ms/step - loss: 0.5551 - accuracy: 0.3523
Epoch 5/100
21/21 [==============================] - 2s 88ms/step - loss: 0.5468 - accuracy: 0.3778
Epoch 6/100
21/21 [==============================] - 2s 90ms/step - loss: 0.5263 - accuracy: 0.3943
Epoch 7/100
21/21 [==============================] - 2s 103ms/step - loss: 0.5442 - accuracy: 0.3688
Epoch 8/100
21/21 [==============================] - 2s 95ms/step - loss: 0.5243 - accuracy: 0.3898
Epoch 9/100
21/21 [==============================] - 2s 106ms/step - loss: 0.4993 - accuracy: 0.4768
Epoch 10/100
21/21 [==============================] - 2s 97ms/step - loss: 0.5034 - accuracy: 0.47